In [ ]:
import cadquery as cq
import numpy as np
from jupyter_cadquery.cadquery import show
from jupyter_cadquery import set_defaults, set_sidecar
from jupyter_cadquery.cadquery import PartGroup, Part, Edges, Faces, Vertices
from jupyter_cadquery.cad_animation import Animation


set_defaults(display='sidecar', axes=False, axes0=True, grid=False,
             default_color='lightgrey', tools=True, ortho=True, black_edges=True)

from cq_gears import (SpurGear, HerringboneGear, RackGear, HerringboneRackGear,
                      PlanetaryGearset, HerringbonePlanetaryGearset,
                      BevelGearPair, Worm, CrossedGearPair, HyperbolicGearPair)

set_sidecar('Gears', init=True)

In [ ]:
L = lambda *args: cq.Location(cq.Vector(*args))
Lrz = lambda loc, r: cq.Location(cq.Vector(*loc), cq.Vector(0.0, 0.0, 1.0), r)
C = lambda *args: cq.Color(*args)

## Parallel axes gears

In [ ]:
def animate_pinion_and_gear(pinion, gear):
    ratio = pinion.z / gear.z

    asm = (cq.Assembly(name='gears')
           .add(pinion.build(), name='pinion', color=C('goldenrod'),
                loc=L(0.0, 0.0, 0.0))
           .add(gear.build(), name='gear', color=C('lightsteelblue'),
                loc=L(gear.r0 + pinion.r0, 0.0, 0.0)))

    view = show(asm)

    n_frames = 800

    times = np.linspace(0.0, 10.0, n_frames)
    pinion_angles = np.linspace(0.0, 360.0, n_frames)
    gear_angles = -pinion_angles * ratio

    anim = Animation(view.root_group)

    anim.add_track(f'gears/pinion', 'rz', times, pinion_angles)
    anim.add_track(f'gears/gear', 'rz', times, gear_angles)

    return anim.animate(autoplay=True)

### Spur gears

In [ ]:
pinion = SpurGear(module=2.0, teeth_number=17, width=10.0, helix_angle=0.0,
                  bore_d=10.0, chamfer=0.5)
gear = SpurGear(module=2.0, teeth_number=29, width=10.0, helix_angle=0.0,
                bore_d=10.0, chamfer=0.5, n_spokes=4, spokes_id=20.0,
                spokes_od=44.0, spoke_width=6.0, spoke_fillet=4.0)

animate_pinion_and_gear(pinion, gear)

### Helical gears

In [ ]:
pinion = SpurGear(module=2.0, teeth_number=17, width=10.0, helix_angle=30.0,
                  bore_d=10.0, chamfer=0.5)
gear = SpurGear(module=2.0, teeth_number=29, width=10.0, helix_angle=-30.0,
                bore_d=10.0, chamfer=0.5, n_spokes=4, spokes_id=20.0,
                spokes_od=44.0, spoke_width=6.0, spoke_fillet=4.0)

animate_pinion_and_gear(pinion, gear)

### Herringbone gears

In [ ]:
pinion = HerringboneGear(module=2.0, teeth_number=17, width=10.0,
                         helix_angle=-30.0, bore_d=10.0)
gear = HerringboneGear(module=2.0, teeth_number=29, width=10.0,
                       helix_angle=30.0, bore_d=10.0, n_spokes=4,
                       spokes_id=20.0, spokes_od=44.0, spoke_width=6.0,
                       spoke_fillet=4.0)

animate_pinion_and_gear(pinion, gear)

## Gear rack and pinion

In [ ]:
def animate_rack_and_pinion(rack, pinion):
    cfr = pinion.r0 * 2.0 * np.pi
    rp = cfr / pinion.z
    start_x = rp * 1.0
    end_x = rack.length - rp * 2.0
    start_angle = 360.0 / pinion.z * 2.0 - 90.0 + np.degrees(pinion.tau) / 2.0
    end_angle = (end_x - start_x) / cfr * 360.0
    
    asm = (cq.Assembly(name='gears')
           .add(rack.build(), name='rack', color=C('goldenrod'),
                loc=L(0.0, 0.0, 0.0))
           .add(pinion.build(), name='pinion', color=C('lightsteelblue'),
                loc=Lrz((start_x, pinion.r0, 0.0), start_angle))
          )

    view = show(asm)

    n_frames = 800

    times = np.linspace(0.0, 10.0, n_frames)
    pinion_angles = -np.linspace(0.0, end_angle, n_frames // 2)
    pinion_angles = np.concatenate((pinion_angles,
                                -np.linspace(end_angle, 0.0, n_frames // 2)))
    pinion_tx = np.linspace(start_x, end_x, n_frames // 2)
    pinion_tx = np.concatenate((pinion_tx, 
                                np.linspace(end_x, start_x, n_frames // 2)))

    anim = Animation(view.root_group)

    anim.add_track(f'gears/pinion', 'rz', times, pinion_angles)
    anim.add_track(f'gears/pinion', 'tx', times, pinion_tx)

    return anim.animate(autoplay=True)


### Straigh teeth rack and pinion

In [ ]:
pinion = SpurGear(module=2.0, teeth_number=18, width=10.0,
                  bore_d=10.0, chamfer=0.5)
rack = RackGear(module=2.0, length=140.0, width=10.0, height=20.0)

animate_rack_and_pinion(rack, pinion)

### Helical rack and pinion

In [ ]:
pinion = SpurGear(module=2.0, teeth_number=18, width=10.0, helix_angle=30.0,
                  bore_d=10.0, chamfer=0.5)
rack = RackGear(module=2.0, length=140.0, width=10.0, height=20.0,
                helix_angle=-30.0)

animate_rack_and_pinion(rack, pinion)

### Herringbone rack and pinion

In [ ]:
pinion = HerringboneGear(module=2.0, teeth_number=18, width=10.0,
                         helix_angle=30.0, bore_d=10.0, chamfer=0.5)
rack = HerringboneRackGear(module=2.0, length=140.0, width=10.0, height=20.0,
                           helix_angle=-30.0)

animate_rack_and_pinion(rack, pinion)

## Planetary gearsets

In [ ]:
def animate_planetary_gearset(gearset):
    planet_ratio = gearset.sun.z / (gearset.planet.z * 2.0)
    carrier_ratio = 1.0 / (gearset.ring.z / gearset.sun.z + 1.0)

    asm = gearset.assemble()
    view = show(asm)

    n_frames = 4000

    times = np.linspace(0.0, 60.0, n_frames)
    sun_angles = np.linspace(0.0, 360.0 * 4.0, n_frames)
    carrier_angles = sun_angles * carrier_ratio
    planet_angles = -sun_angles * planet_ratio

    anim = Animation(view.root_group)

    anim.add_track(f'planetary/sun', 'rz', times, sun_angles)
    anim.add_track(f'planetary/planets', 'rz', times, carrier_angles)

    for i in range(gearset.n_planets):
        anim.add_track(f'planetary/planets/planet_{i:02}', 'rz', times,
                       planet_angles - carrier_angles)

    return anim.animate(autoplay=True)

### Straight teeth planetary gearset

In [ ]:
gearset = PlanetaryGearset(module=1.0,
                           sun_teeth_number=40, planet_teeth_number=20,
                           width=10.0, rim_width=5.0, n_planets=6,
                           bore_d=6.0, chamfer=0.5,
                           sun_build_args={'bore_d': 20.0})

animate_planetary_gearset(gearset)

### Helical planetary gearset

In [ ]:
gearset = PlanetaryGearset(module=1.0, helix_angle=30.0,
                           sun_teeth_number=40, planet_teeth_number=20,
                           width=10.0, rim_width=5.0, n_planets=6,
                           bore_d=6.0, chamfer=0.5,
                           sun_build_args={'bore_d': 20.0})

animate_planetary_gearset(gearset)

### Herringbone planetary gearset

In [ ]:
gearset = HerringbonePlanetaryGearset(
                           module=1.0, helix_angle=30.0,
                           sun_teeth_number=40, planet_teeth_number=20,
                           width=10.0, rim_width=5.0, n_planets=6,
                           bore_d=6.0, chamfer=0.5,
                           sun_build_args={'bore_d': 20.0})

animate_planetary_gearset(gearset)

## Bevel gears

In [ ]:
def animate_bevel_gear_pair(gearset):
    asm = gearset.assemble(transform_pinion=True)

    view = show(asm)

    n_frames = 4000
    ratio = gearset.pinion.z / gearset.gear.z

    times = np.linspace(0.0, 60.0, n_frames)
    pinion_angles = np.linspace(0.0, 360.0 * 4.0, n_frames)
    gear_angles = -pinion_angles * ratio

    anim = Animation(view.root_group)

    anim.add_track(f'bevel_pair/pinion', 'rz', times, pinion_angles)
    anim.add_track(f'bevel_pair/gear', 'rz', times, gear_angles)

    return anim.animate(autoplay=True)

### Straight bevel gears

In [ ]:
gearset = BevelGearPair(module=2.0, gear_teeth=23, pinion_teeth=14,
                        face_width=12.0, axis_angle=90.0, bore_d=12.0,
                        pinion_build_args={'bore_d': 8.0})

animate_bevel_gear_pair(gearset)

### Helical bevel gears

In [ ]:
gearset = BevelGearPair(module=2.0, gear_teeth=23, pinion_teeth=14,
                        face_width=12.0, helix_angle=30.0, axis_angle=90.0,
                        bore_d=12.0, pinion_build_args={'bore_d': 8.0})

animate_bevel_gear_pair(gearset)

## Worm gears

In [ ]:
def animate_worm_and_gear(worm, gear):
    ratio = worm.n_threads / gear.z
    align_angle = -90.0 - np.degrees(gear.tau / 2.0 - gear.twist_angle / 2.0)

    n_frames = 1000
    times = np.linspace(0.0, 60.0, n_frames)
    worm_angles = np.linspace(0.0, 360.0 * 10.0, n_frames)
    gear_angles = worm_angles * ratio

    asm = (cq.Assembly(name='gears')
           .add(worm.build(), name='worm', color=C('lightsteelblue'),
                loc=L(0.0, 0.0, worm.r0))
           .add(gear.build(), name='gear', color=C('goldenrod'),
                loc=Lrz((0.0,
                         worm.r0 + gear.r0,
                         worm.r0 - gear.width / 2.0), align_angle)))

    view = show(asm)

    anim = Animation(view.root_group)
    anim.add_track('gears/worm', 'rx', times, worm_angles)
    anim.add_track('gears/gear', 'rz', times, gear_angles)

    return anim.animate(autoplay=True)
    

### Worm and gear (non-throated)

In [ ]:
lead_angle = 7.0
worm = Worm(module=2.0, lead_angle=lead_angle, n_threads=2, length=60.0,
            bore_d=20.0)
gear = SpurGear(module=2.0, teeth_number=32, width=16.0, helix_angle=lead_angle,
                chamfer=0.5, bore_d=10.0, recess_d=50.0, recess=6.0,
                hub_d=20.0, hub_length=5.0)

animate_worm_and_gear(worm, gear)

## Crossed axes helical gears

In [ ]:
def animate_crossed_gear_pair(gearset):
    asm = gearset.assemble()

    view = show(asm)

    n_frames = 1000
    ratio = gearset.gear1.z / gearset.gear2.z

    times = np.linspace(0.0, 60.0, n_frames)
    gear1_angles = np.linspace(0.0, 360.0 * 4.0, n_frames)
    gear2_angles = -gear1_angles * ratio

    anim = Animation(view.root_group)

    anim.add_track('crossed_pair/gear1', 'rz', times, gear1_angles)
    anim.add_track('crossed_pair/gear2', 'rz', times, gear2_angles)

    return anim.animate(autoplay=True)

In [ ]:
gears = CrossedGearPair(module=2.0,
                        gear1_teeth_number=19, gear2_teeth_number=19,
                        gear1_width=16.0, gear2_width=40.0,
                        gear1_helix_angle=40.0, shaft_angle=90.0,
                        bore_d=16.0, chamfer=1.0)

animate_crossed_gear_pair(gears)

## Hyperbolic gears

In [ ]:
def animate_hyperbolic_gear_pair(gearset):
    asm = gearset.assemble()

    view = show(asm)

    n_frames = 1000
    ratio = gearset.gear1.z / gearset.gear2.z

    times = np.linspace(0.0, 60.0, n_frames)
    gear1_angles = np.linspace(0.0, 360.0 * 4.0, n_frames)
    gear2_angles = -gear1_angles * ratio

    anim = Animation(view.root_group)

    anim.add_track('hyperbolic_pair/gear1', 'rz', times, gear1_angles)
    anim.add_track('hyperbolic_pair/gear2', 'rz', times, gear2_angles)

    return anim.animate(autoplay=True)

In [ ]:
gears = HyperbolicGearPair(module=2.0, gear1_teeth_number=13, width=60.0,
                           shaft_angle=42.0, gear2_teeth_number=13,
                           bore_d=8.0, chamfer=(4.0, 8.0))

animate_hyperbolic_gear_pair(gears)